## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# Cast IS_SUCCESSFUL and STATUS columns as boolean for clarity
application_df['IS_SUCCESSFUL'] = application_df['IS_SUCCESSFUL'].astype(bool)
application_df['STATUS'] = application_df['STATUS'].astype(bool)
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                      bool
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL               bool
dtype: object

In [5]:
# Determine the number of unique values in each column.
# First limit to only columns with object datatype
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Check number of unique values for the object columns only
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_counts = classification_counts[classification_counts > 1]
filtered_classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace
cutoff_value = 1883
classifications_to_replace = []
for classification, count in classification_counts.items():
  if count < cutoff_value:
    classifications_to_replace.append(classification)

# Look at the list of classifications to replace
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [12]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_transformed = pd.get_dummies(application_df)

# Look at results
application_df_transformed.dtypes

STATUS                           bool
ASK_AMT                         int64
IS_SUCCESSFUL                    bool
APPLICATION_TYPE_Other           bool
APPLICATION_TYPE_T10             bool
APPLICATION_TYPE_T19             bool
APPLICATION_TYPE_T3              bool
APPLICATION_TYPE_T4              bool
APPLICATION_TYPE_T5              bool
APPLICATION_TYPE_T6              bool
APPLICATION_TYPE_T7              bool
APPLICATION_TYPE_T8              bool
APPLICATION_TYPE_T9              bool
AFFILIATION_CompanySponsored     bool
AFFILIATION_Family/Parent        bool
AFFILIATION_Independent          bool
AFFILIATION_National             bool
AFFILIATION_Other                bool
AFFILIATION_Regional             bool
CLASSIFICATION_C1000             bool
CLASSIFICATION_C1200             bool
CLASSIFICATION_C2000             bool
CLASSIFICATION_C2100             bool
CLASSIFICATION_C3000             bool
CLASSIFICATION_Other             bool
USE_CASE_CommunityServ           bool
USE_CASE_Hea

In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df_transformed['IS_SUCCESSFUL'].values
X = application_df_transformed.drop(['IS_SUCCESSFUL'], axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Find the shape of the IS_SUCCESSFUL data
y.shape

(34299,)

In [16]:
# Find the shape of the remaining data
X.shape

(34299, 44)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Find out how many columns are in the X data
len(X_train[0])

44

## Model 1 - Compile, Train and Evaluate the Model



In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = 44, activation = "leaky_relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3600      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6061 (23.68 KB)
Trainable params: 6061 (23.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5769 - accuracy: 0.7198
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5591 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5556 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5538 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7318
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accura

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5526 - accuracy: 0.7224 - 494ms/epoch - 2ms/step
Loss: 0.5525864362716675, Accuracy: 0.722449004650116


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityModel1.h5")

## Model 2 - Compile, Train and Evaluate the Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = 44, activation = "leaky_relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 50, activation = "leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3600      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 50)                1550      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7631 (29.81 KB)
Trainable params: 7631 (29.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5702 - accuracy: 0.7242
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5488 - accuracy: 0.7333
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7330
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7269 - 664ms/epoch - 2ms/step
Loss: 0.554524302482605, Accuracy: 0.7268804907798767


In [30]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityModel2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Model 3 - Compile, Train and Evaluate the Model



In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 30, input_dim = 44, activation = "leaky_relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 30)                1350      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1671 (6.53 KB)
Trainable params: 1671 (6.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5910 - accuracy: 0.7112
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5597 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5526 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7314
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7317
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accura

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5542 - accuracy: 0.7278 - 448ms/epoch - 2ms/step
Loss: 0.5542190074920654, Accuracy: 0.7278134226799011


In [36]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityModel3.h5")